In [10]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import radiomics
import SimpleITK as sitk
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from RadiomicsDataset import RadiomicsDataset
from SegmentationDataset import SegmentationDataset
import albumentations as A
import torchmetrics
from torchsummary import summary
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from torchvision import transforms

radiomics.logger.setLevel(40)

In [5]:
glcm_feats = keys_list = [
    # 'original_glcm_Autocorrelation',
    'Autocorrelation',
    'ClusterProminence',
    'ClusterShade',
    'ClusterTendency',
    'Contrast',
    'Correlation',
    'DifferenceAverage',
    'DifferenceEntropy',
    'DifferenceVariance',
    'Id',
    'Idm',
    'Idmn',
    'Idn',
    'Imc1',
    'Imc2',
    'InverseVariance',
    'JointAverage',
    'JointEnergy',
    'JointEntropy',
    'MCC',
    'MaximumProbability',
    # 'SumAverage',
    'SumEntropy',
    'SumSquares'
]

In [11]:
# load one image and mask as a sample as numpy array
image_path = "dataset/benign/benign (7).png"
mask_path = "dataset/benign/benign (7)_mask.png"

# Configure the feature extractor
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor()
# extractor.enableAllFeatures()
# extractor.disableAllFeatures()
# extractor.enableFeatureClassByName('firstorder')

extractor.enableAllFeatures()
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('shape2D')
extractor.enableFeaturesByName(glcm=glcm_feats)
extractor.enableFeatureClassByName('gldm')
extractor.enableFeatureClassByName('glrlm')
extractor.enableFeatureClassByName('glszm')
extractor.enableFeatureClassByName('ngtdm')
# extractor.enableFeatureClassByName()

image = sitk.ReadImage(image_path, sitk.sitkUInt32)
mask = sitk.ReadImage(mask_path, sitk.sitkUInt8)
features = extractor.execute(image, mask, voxelBased=False, label=255)

In [12]:
clean_features_dict = {key: float(features[key]) for key in features if key.startswith('original_')}
clean_features_dict # these are they keys features that we will work with

{'original_shape2D_Elongation': 0.7636137691555834,
 'original_shape2D_MajorAxisLength': 60.80479223753312,
 'original_shape2D_MaximumDiameter': 62.39390995922599,
 'original_shape2D_MeshSurface': 2203.5,
 'original_shape2D_MinorAxisLength': 46.431376583224825,
 'original_shape2D_Perimeter': 192.1248916810282,
 'original_shape2D_PerimeterSurfaceRatio': 0.08719078360836315,
 'original_shape2D_PixelSurface': 2204.0,
 'original_shape2D_Sphericity': 0.8661195144977577,
 'original_firstorder_10Percentile': 32.0,
 'original_firstorder_90Percentile': 130.0,
 'original_firstorder_Energy': 15620550.0,
 'original_firstorder_Entropy': 2.549458989649134,
 'original_firstorder_InterquartileRange': 49.0,
 'original_firstorder_Kurtosis': 4.785778136404155,
 'original_firstorder_Maximum': 237.0,
 'original_firstorder_MeanAbsoluteDeviation': 32.873800646242934,
 'original_firstorder_Mean': 72.4328493647913,
 'original_firstorder_Median': 60.0,
 'original_firstorder_Minimum': 13.0,
 'original_firstorder

In [8]:
len(clean_features_dict)

101